# Traffic Sign Classification

bron: https://www.kaggle.com/niranjankumarc/traffic-sign-classification-keras-tensorflow-2-0/data


![traffic_sign_classification_dataset.jpg](images/traffic_sign_classification_dataset.jpg)



In [30]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os


#import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, Activation, Flatten, Dropout
from tensorflow.keras.layers import Dense

In [31]:
# check tensorflow version
tf.__version__

'2.0.0'

In [32]:
#create a custom class to train the model.

class TrafficSignNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        #layer: Conv -> RELU -> BN -> POOL
        model.add(Conv2D(8, (5,5), padding = "same", input_shape = inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (2,2)))
        
        # first set of (CONV => RELU => CONV => RELU) * 2 => POOL
        model.add(Conv2D(16, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(Conv2D(16, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (2,2)))
        
        # second set of (CONV => RELU => CONV => RELU) * 2 => POOL
        model.add(Conv2D(32, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(Conv2D(32, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(Conv2D(32, (3,3), padding = "same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis = chanDim))
        model.add(MaxPooling2D(pool_size = (2,2)))
        
        #FC layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.7))
        
        #FC layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        #softmax
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [33]:
#create a class object

objTSN = TrafficSignNet()

In [34]:
#import necessary packages

import matplotlib.pyplot as plt
plt.style.use("seaborn")

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from skimage import transform, exposure, io
import random
import os

We can automatically improve image contrast by applying an algorithm called Contrast Limited Adaptive Histogram Equalization (CLAHE), the implementation of which can be found in the scikit-image library.

Using CLAHE we can improve the contrast of our traffic sign images.

In [35]:
#define a function to load data from disk

def load_split(basePath, csvPath):
    #intialize the list of data and labels
    data = []
    labels = []
    
    # load the contents of the CSV file, remove the first line (since it contains the CSV header)
    rows = open(csvPath).read().strip().split("\n")[1:]
    random.shuffle(rows)
    
    #loop over the rows of csv file
    for (i, row) in enumerate(rows):
        #check to see if we should show a status update
        if i > 0 and i % 4000 == 0:
            print("[INFO] processed {} total images".format(i))
            
        # split the row into components and then grab the class ID and image path
        (label, imagePath) = row.strip().split(",")[-2:]
        
        # derive the full path to the image file and load it
        imagePath = os.path.sep.join([basePath, imagePath])
        #print(imagePath)
        image = io.imread(imagePath)
        
        #resize the image to be 32x32 pixels, ignoring aspect ratio, and perform CLAHE.
        image = transform.resize(image, (32, 32))
        image = exposure.equalize_adapthist(image, clip_limit = 0.1)
        
        #update the list of data and labels, respectively
        data.append(image)
        labels.append(int(label))
        
    #convert the data and labels into numpy arrays
    data = np.array(data)
    labels = np.array(labels)
    
    #return a tuple of the data and labels
    return (data, labels)

In [36]:
# initialize the hyperparameters: 
#      initial learning rate, batch size, and number of epochs to train for

INIT_LR = 1e-3
BS = 64
NUM_EPOCHS = 50

#load the label names

labelNames = ['20 km/h', '30 km/h', '50 km/h', '60 km/h', '70 km/h', '80 km/h', '80 km/h end', '100 km/h', '120 km/h', 'No overtaking',
               'No overtaking for tracks', 'Crossroad with secondary way', 'Main road', 'Give way', 'Stop', 'Road up', 'Road up for track', 'Brock',
               'Other dangerous', 'Turn left', 'Turn right', 'Winding road', 'Hollow road', 'Slippery road', 'Narrowing road', 'Roadwork', 'Traffic light',
               'Pedestrian', 'Children', 'Bike', 'Snow', 'Deer', 'End of the limits', 'Only right', 'Only left', 'Only straight', 'Only straight and right', 
               'Only straight and left', 'Take right', 'Take left', 'Circle crossroad', 'End of overtaking limit', 'End of overtaking limit for track']


In [37]:
# derive the path to the training and testing CSV files

path = './datasets/gtsrb-german-traffic-sign'  # 2019-1216 PP added

trainPath = os.path.sep.join([path, "Train.csv"])
testPath = os.path.sep.join([path, "Test.csv"])

trainPath

'./datasets/gtsrb-german-traffic-sign/Train.csv'

In [38]:
# load the training and testing data
# 2019-1216 duurt lang!!

print("[INFO] loading training and testing data...")

(trainX, trainY) = load_split(path, trainPath)
(testX, testY) = load_split(path, testPath)

[INFO] loading training and testing data...
[INFO] processed 4000 total images
[INFO] processed 8000 total images
[INFO] processed 12000 total images
[INFO] processed 16000 total images
[INFO] processed 20000 total images
[INFO] processed 24000 total images
[INFO] processed 28000 total images
[INFO] processed 32000 total images
[INFO] processed 36000 total images
[INFO] processed 4000 total images
[INFO] processed 8000 total images
[INFO] processed 12000 total images


In [13]:
#normalize the images

trainX = trainX.astype("float32")/255.0
testX = testX.astype("float32")/255.0

In [14]:
#one hot encoding of labels

numLabels = len(np.unique(trainY))
trainY = to_categorical(trainY, numLabels)
testY = to_categorical(testY, numLabels)

#take class weight into account
classTotals = trainY.sum(axis = 0)
classWeight = classTotals.max()/classTotals

In [15]:
classWeight

array([[  1.       , 185.70952  ],
       [  1.0543405,  17.567118 ],
       [  1.0551963,  17.33289  ],
       [  1.0317469,  27.658865 ],
       [  1.0475435,  19.696465 ],
       [  1.0441779,  20.967205 ],
       [  1.0054139,  92.85476  ],
       [  1.0325664,  27.08264  ],
       [  1.0317469,  27.658865 ],
       [  1.0333872,  26.529932 ],
       [  1.0483884,  19.402487 ],
       [  1.0292962,  29.544697 ],
       [  1.0509311,  18.570951 ],
       [  1.052633 ,  18.055092 ],
       [  1.0148325,  49.99872  ],
       [  1.0108868,  61.903175 ],
       [  1.0054139,  92.85476  ],
       [  1.0236226,  35.134235 ],
       [  1.0260465,  32.49917  ],
       [  1.       , 185.70952  ],
       [  1.0038611, 108.33056  ],
       [  1.0030864, 118.17879  ],
       [  1.0046369,  99.99744  ],
       [  1.0077522,  76.46863  ],
       [  1.0015409, 144.44073  ],
       [  1.0342094,  25.999332 ],
       [  1.0101013,  64.99834  ],
       [  1.0007699, 162.49583  ],
       [  1.008534 ,

In [28]:
#construct the image data augmentation generator
# lees meer: 
#     https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/
#     dd. 8 juni 2019

aug = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.15,
    width_shift_range = 0.1,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = False,
    vertical_flip = False,
    fill_mode = "nearest"
)


In [29]:
# initialize the optimizer and compile the model

print("[INFO] compiling model...")
opt = Adam(lr = INIT_LR, decay = INIT_LR/(NUM_EPOCHS * 0.5))
# 2019-1217 TEST: from tensorflow.keras.optimizers import SGD
# 2019-1217 TEST: opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS * 0.5)

model = objTSN.build(width = 32, height = 32, depth = 3, classes=numLabels)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])


[INFO] compiling model...


## Train the model

In [17]:
#train the network

H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size = BS),
    validation_data = (testX, testY),
    steps_per_epoch = trainX.shape[0]//BS,
    epochs = NUM_EPOCHS,
    class_weight = classWeight,
    verbose = 1
)


Epoch 1/50


ValueError: A target array with shape (64, 43, 2) was passed for an output of shape (None, 2) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#evaluate network
predictions = model.predict(testX, batch_size = BS)
print(classification_report(testY.argmax(axis = 1), predictions.argmax(axis = 1), target_names = labelNames))

#save the model
model.save("trainedmodel.h5")


In [ ]:
#plot the training loss and accuracy

N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label = "train_loss")
plt.plot(N, H.history["val_loss"], label = "val_loss")
plt.plot(N, H.history["accuracy"], label = "accuracy")
plt.plot(N, H.history["val_accuracy"], label = "val_acc")
plt.title("Training Loss and Accuracy on DataSet")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc = "lower left")
plt.savefig("plot.png")
